In [1]:
import pandas as pd

In [3]:
def csv_to_df(file_path:str):
    """
    Takes in Relative Path for CSV file
    Returns pandas dataframe
    """
    return pd.read_csv(file_path)


In [14]:
df = csv_to_df("../data/2021-SxS-Data-and-Data-Info/partial_cleaned_2021.csv")

In [24]:
def gen_big_3_df(df):
    feature_df = df[['state','math_conf_lvl','fin_sat_lvl','fin_conf_lvl']]

    # Define the columns and values to replace
    columns_dict = {
        'math_conf_lvl': [99, 98],
        'fin_sat_lvl': [99, 98],
        'fin_conf_lvl': [99, 98],
   
    }
    # Loop over the dictionary to replace the values
    for column, values in columns_dict.items():
        for value in values:
            feature_df.loc[feature_df[column] == value, column] = None
    feature_df = feature_df.dropna()
    return feature_df

In [23]:
def gen_geo_df(mini_df):
    state_math_avg =df[['state','math_conf_lvl']].groupby('state').mean().reset_index()
    state_math_avg=state_math_avg.sort_values(by='state')

    state_sat_avg =df[['state','fin_sat_lvl']].groupby('state').mean().reset_index()
    state_sat_avg=state_sat_avg.sort_values(by='state')

    fin_conf_avg = df[['state','fin_conf_lvl']].groupby('state').mean().reset_index()
    fin_conf_avg = fin_conf_avg.sort_values(by='state')
    # Load file
    url = "https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/us_states_hexgrid.geojson.json"
    geoData = gpd.read_file(url)
    # Add a new column to the geo dataframe that will be used for joining:
    geoData['state'] = geoData['google_name'].str.replace('(United States)','')
    # Sort by column: 'state' (ascending)
    geoData = geoData.sort_values(['state']).reset_index()
    # Drop column: 'bees'
    geoData = geoData.drop(columns=['bees', 'index'])
    #Create Columns from subsets
    geoData['fin_sat_avg'] = state_sat_avg['fin_sat_lvl']
    geoData['math_conf_avg'] = state_math_avg['math_conf_lvl']
    geoData['fin_conf_avg'] = fin_conf_avg['fin_conf_lvl']
    return geoData


,state,math_conf_lvl,fin_sat_lvl,fin_conf_lvl
0,South Carolina,6.0,9.0,7.0
1,Ohio,1.0,4.0,1.0
2,Arizona,7.0,10.0,7.0
3,Arizona,6.0,9.0,7.0
4,Ohio,6.0,4.0,6.0
...,...,...,...,...
27113,Idaho,6.0,1.0,4.0
27114,Hawaii,7.0,1.0,3.0
27115,Hawaii,5.0,7.0,7.0
27116,District of Columbia,6.0,1.0,5.0


In [12]:
def gen_hexbin_map(option):
    """
    Generated hexbin map based on user selection
    """
    return f'You picked {option}'

In [13]:
gen_hexbin_map('cookie')

'You picked  cookie'